In [1]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"

## asyncio AI communication

In [ ]:
from collections import deque
import asyncio
import nest_asyncio
nest_asyncio.apply()

class Agent:
    def __init__(self, name: str, state: int, max_round: int):
        # initialize the basic info
        self.name = name
        self.state = state
        self.inbox = asyncio.PriorityQueue()
        self.sent = asyncio.PriorityQueue()
        self.agents = {}
        self.round = 0
        self.max_round = max_round


    async def send_message(self, other_agent, message: int, priority):
        await self.sent.put((priority, (other_agent.name, message)))
        await other_agent.inbox.put((priority, (self.name, message)))

    async def process_message(self):
        while self.round <= self.max_round:
          priority, full_message = await self.inbox.get()
          sender = full_message[0]
          message = full_message[1]
          self.state += message
          print(f"I (Agent {self.name}) Received messages {message} from Agent {sender}. Now my state is {self.state}")
          self.round += 1
          await self.send_message(self.agents[sender], self.state, priority)

async def main():
  agent1 = Agent('Alice', 1, 5)
  agent2 = Agent('Bob', 2, 5)
  agent1.agents[agent2.name] = agent2
  agent2.agents[agent1.name] = agent1

  task1 = asyncio.create_task(agent1.send_message(agent2, 10, 2))
  task2 = asyncio.create_task(agent1.send_message(agent2, 5, 1))
  task3 = asyncio.create_task(agent1.process_message())
  task4 = asyncio.create_task(agent2.process_message())

  await asyncio.sleep(2)

asyncio.run(main())

I (Agent Bob) Received messages 5 from Agent Alice. Now my state is 7
I (Agent Bob) Received messages 10 from Agent Alice. Now my state is 17
I (Agent Alice) Received messages 7 from Agent Bob. Now my state is 8
I (Agent Alice) Received messages 17 from Agent Bob. Now my state is 25
I (Agent Bob) Received messages 8 from Agent Alice. Now my state is 25
I (Agent Bob) Received messages 25 from Agent Alice. Now my state is 50
I (Agent Alice) Received messages 25 from Agent Bob. Now my state is 50
I (Agent Alice) Received messages 50 from Agent Bob. Now my state is 100
I (Agent Bob) Received messages 50 from Agent Alice. Now my state is 100
I (Agent Bob) Received messages 100 from Agent Alice. Now my state is 200
I (Agent Alice) Received messages 100 from Agent Bob. Now my state is 200
I (Agent Alice) Received messages 200 from Agent Bob. Now my state is 400


## execute code module

In [4]:
from openai import OpenAI
import ast

def llm_parse_math_problem(problem):
    """ Use LLM to parse a math problem and convert it into Python code """
    prompt = f"Convert the following math problem into Python code. Only output the code block, and name the final result variable as 'result':\n\n{problem}"

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a math-solving assistant"},
                  {"role": "user", "content": prompt}]
    )
    code = response.choices[0].message.content
    clean_code = code.strip("```python").strip("```")

    return clean_code

def execute_math_code(code):
    """ Execute the Python code generated by LLM """
    try:
        tree = ast.parse(code, mode='exec')  # Ensure the code is executable
        exec_globals = {}
        exec(code, exec_globals)
        return exec_globals.get("result", "Undefined result")
    except Exception as e:
        return f"Code execution error: {str(e)}"

def score_answer(question, answer):
    """ Scoring system: Use LLM to evaluate the correctness of the answer """
    # Use a CoT prompt to ensure the LLM can do the math correctly.
    prompt = f"""
    Check whether the answer to the following math problem is correct by following a step-by-step reasoning process. First, break down the problem and solve it step by step. Then, compare the computed result with the provided answer. Finally, return 'Correct' if they match, or 'Incorrect' otherwise.

    Example:

    Problem: Solve 3x + 7 = 16. What is the value of x?
    Answer: 4

    Step-by-step reasoning:
    1. Start with the equation: 3x + 7 = 16
    2. Subtract 7 from both sides: 3x = 9
    3. Divide both sides by 3: x = 3
    4. The computed result is 3, but the provided answer is 4.
    5. Since 3 ≠ 4, the answer is incorrect.

    Final verdict: Incorrect

    Now, apply the same reasoning to the following problem.

    Problem: {question}
    Answer: {answer}

    Provide a step-by-step solution and return either 'Correct' or 'Incorrect'."""

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a grader"},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


# Test example
math_problem = "Solve 2x + 5 = 15. What is the value of x?"

# Step 1: LLM parses the math problem
generated_code = llm_parse_math_problem(math_problem)
print("*******LLM Generated Code:*******\n", generated_code)

# Step 2: Execute the code
math_answer = execute_math_code(generated_code)
print("*******Computed Result:*******\n", math_answer)

# Step 3: Evaluate the answer using the scoring model
evaluation = score_answer(math_problem, math_answer)
print("*******Evaluation Result:*******\n", evaluation)

*******LLM Generated Code:*******
 
import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = sp.Eq(2*x + 5, 15)

# Solve the equation
result = sp.solve(equation, x)[0]

*******Computed Result:*******
 5
*******Evaluation Result:*******
 Step-by-step reasoning:  
1. Start with the equation: 2x + 5 = 15  
2. Subtract 5 from both sides: 2x = 10  
3. Divide both sides by 2: x = 5  
4. The computed result is 5, and the provided answer is also 5.  
5. Since 5 = 5, the answer is correct.  

Final verdict: Correct  


## multi-agent collaboration

(1) code execution\
(2) Tool using (like google API?)\
(3) communication module\
(4) plan module\
(5) RAG module


In [ ]:
import re
import json

class Agent:
  def __init__(self, name, profile):
    self.name = name
    self.profile = profile
    self.response = None
    self.friends = {}
    self.inbox = []
    self.sent = []
    self.round = 0
    self.max_round = 2
    self.position = []

  def send_message(self, receiver, message):
    self.sent.append((receiver.name, message))
    print(f"{self.name} send a message to {receiver.name}. (message: {message})")
    receiver.inbox.append((self.name, message))

  def process_message(self):
    client = OpenAI()
    for key, value in self.friends.items():
      if key not in self.position:
        self.position.append(key)

    if self.inbox and self.round <= self.max_round * 2:
      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[{"role": "system", "content": f"This is your profile: {self.profile}."
                                                  f"Your team members are: {self.position}."
                                                  """Collabrate with your team members to solve your tasks that the manager assigned to you. Return your response. And return your communication message in the valid json format: {"Alice": "please write code for me to solve this problem."} Remember to replace 'Alice' with your real team member's name."""} ,
                    {"role": "user", "content": f"This is your task{self.inbox}"}]
          )
      clean_response = response.choices[0].message.content
      print(f'----------message from {self.name}----------')
      print(f"As a {self.name}, my answer to this question is: {clean_response}")
      self.round += 1
      tasks = self.parse_response(clean_response)
      if tasks:
        self.communicate(tasks)

  def parse_response(self, response):
    communication = re.search(r"\{.*\}", response, re.DOTALL)
    if communication:
      return json.loads(communication.group(0))
    return None

  def communicate(self, task_dict):
    for agent, task in task_dict.items():
      self.send_message(self.friends[agent], task)


In [ ]:
from openai import OpenAI

class Planner(Agent):
  def __init__(self, name, profile, task):
    super().__init__(name, profile)
    self.task = task
    self.teams = {}
    self.progress = None

  def generate_plan(self):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": """You're a planner. You'll receive a task, and team members' profile. Assign tasks to them. Return your plan in the valid json format: {"Alice": "do the literature review", "Bob": "write code"}"""},
                  {"role": "user", "content": f"Your task is: {self.task}. Your team members are: {self.friends['Thinker'].name} and {self.friends['Programmer'].name}"}]
        )
    clean_plan = response.choices[0].message.content
    tasks = self.parse_response(clean_plan)
    self.communicate(tasks)


In [ ]:
class programmer(Agent):
  def __init__(self, name, profile):
    super().__init__(name, profile)


In [ ]:
class Engine:
  def __init__(self, planner_name):
    self.agents = {}
    self.planner_name = planner_name

  def add_agent(self, agent):
    self.agents[agent.name] = agent

  def add_friends(self, friends):
    for agent, friend in friends.items():
      for f in friend:
        self.agents[agent].friends[f] = self.agents[f]

  def run(self):
    planner = self.agents[self.planner_name]
    planner.generate_plan()

    for name, agent in self.agents.items():
      agent.process_message()

In [ ]:
engine = Engine('Boss')

planner = Planner(name = "Boss", profile = "You're a planner.", task = "2x + 5 = 15, what is x?")
CoT = Agent("Thinker", "You will be given a question. Think about it in a step by step way.")
programmer = Agent("Programmer", "You will be give a question. Solve it by writing python code. Only return the code!")

engine.add_agent(planner)
engine.add_agent(CoT)
engine.add_agent(programmer)

engine.add_friends({'Boss': ['Thinker', 'Programmer'],
                    'Thinker': ['Programmer'],
                    'Programmer': ['Thinker']})

engine.run()

Boss send a message to Thinker. (message: solve for x in the equation 2x + 5 = 15)
Boss send a message to Programmer. (message: write a program to compute x)
----------message from Thinker----------
As a Thinker, my answer to this question is: To solve the equation 2x + 5 = 15 for x, I will follow these steps:

1. **Isolate the term with x**: Subtract 5 from both sides of the equation.
2. **Perform the subtraction**: This will give us 2x = 15 - 5.
3. **Simplify**: So now we have 2x = 10.
4. **Solve for x**: Divide both sides by 2, which results in x = 10 / 2.
5. **Final value**: Thus, x = 5.

Now, I will ask my Programmer team member to write code to solve similar equations programmatically.

Here's my communication message to my team member formatted in JSON:

{"Programmer": "please write code for me to solve this problem."}
Thinker send a message to Programmer. (message: please write code for me to solve this problem.)
----------message from Programmer----------
As a Programmer, my a

## Graph of Thoughts

In [5]:
import networkx as nx
import openai

class GraphOfThoughts:
    def __init__(self):
        self.graph = nx.DiGraph()
        self.llm_client = openai.Client()  # Connect to OpenAI API

    def add_thought(self, node_id, thought, parent_id=None):
        """Add a new thought node"""
        self.graph.add_node(node_id, thought=thought)
        if parent_id is not None:
            self.graph.add_edge(parent_id, node_id)  # Connect to the previous thought step

    def generate_thought(self, previous_thought):
        """Generate a new thought using LLM"""
        response = self.llm_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You're an AI problem solver."},
                {"role": "user", "content": f"Given the previous thought: '{previous_thought}', what should be the next step in solving the problem?"}
            ]
        )
        return response.choices[0].message.content

    def expand_graph(self, start_node):
        """Recursively expand the thought graph using LLM"""
        stack = [start_node]
        node_id = 1

        while stack and node_id <= 5:  # Limit reasoning depth to 5 levels
            current_node = stack.pop()
            previous_thought = self.graph.nodes[current_node]["thought"]

            # Generate a new thought step
            new_thought = self.generate_thought(previous_thought)
            self.add_thought(node_id, new_thought, current_node)
            stack.append(node_id)
            node_id += 1

    def print_graph(self):
        """Print the thought process"""
        for node, data in self.graph.nodes(data=True):
            print(f"Node {node}: {data['thought']}")

# Initialize GoT
got = GraphOfThoughts()

# Add the initial thought (problem)
got.add_thought(0, "Solve 2x + 5 = 15")

# Expand the thought path
got.expand_graph(0)

# Output the thought process
got.print_graph()

Node 0: Solve 2x + 5 = 15
Node 1: To solve the equation \(2x + 5 = 15\), the next step is to isolate the term containing \(x\). You can do this by subtracting 5 from both sides of the equation:

\[
2x + 5 - 5 = 15 - 5
\]

This simplifies to:

\[
2x = 10
\]

Now you can proceed to the next steps, such as dividing by 2 to solve for \(x\).
Node 2: The next step is to isolate \(x\) by dividing both sides of the equation \(2x = 10\) by 2. 

This will give:

\[
\frac{2x}{2} = \frac{10}{2}
\]

This simplifies to:

\[
x = 5
\]

So the solution to the equation \(2x + 5 = 15\) is \(x = 5\).
Node 3: It seems there is a slight confusion in the concluding statement. The initial equation given was \(2x = 10\), and you correctly found that \(x = 5\). However, the statement later refers to solving \(2x + 5 = 15\). 

To solve the equation \(2x + 5 = 15\), the next step after isolating \(x\) would be to subtract 5 from both sides of the equation:

\[
2x + 5 - 5 = 15 - 5
\]

This simplifies to:

\[
2x = 

## ChatBot

planner: assign tasks to different agents\
agents: programmer for code, writer for general response

In [ ]:
class Agent:
  def __init__(self, name, role):
    self.name = name
    self.role = role
    self.inbox = []
    self.sent = []
    self.friends = {}

  def send_message(self, receiver: str, message):
    self.friends[receiver].inbox.append((self.name, message))
    self.sent.append((receiver, message))

  def process_message(self):
    if self.inbox:
      message = self.inbox.pop()
      if self.name == 'planner':
        if 'code' in message:
          self.send_message('programmer', message)
        else:
          self.send_message('writer', message)
      elif self.name == 'programmer':
        response = 'print("a = 100")'
        print("==========programmer response==========")
        print(response)
      else:
        response = 'I have received your input, and we will contact you later!'
        print("==========writer response==========")
        print(response)

  def parse(self):
    pass

  def communicate(self):
    pass

class Engine:
  def __init__(self):
    self.agents = {}

  def add_agents(self, agent):
    self.agents[agent.name] = agent

  def add_friends(self, relationship):
    for me, friends in relationship.items():
      for friend in friends:
        self.agents[me].friends[friend] = self.agents[friend]

  def run(self):
    while True:
      user_input = input(f"Hi! How can I help you? \n")
      if 'exit' in user_input:
        print("==========chatbot response==========")
        print("GoodBye!")
        break
      self.agents['planner'].inbox.append(user_input)
      # while any(agent.inbox for agent in self.agents.values()):
      for agent_name, agent in self.agents.items():
        agent.process_message()


engine = Engine()
planner = Agent('planner', 'To assign tasks.')
writer = Agent('writer', 'To answer general questions.')
programmer = Agent('programmer', 'To write code.')

relationship = {'planner': ['writer', 'programmer']}
engine.add_agents(planner)
engine.add_agents(writer)
engine.add_agents(programmer)
engine.add_friends(relationship)


engine.run()

Hi! How can I help you? 
Hi!
==========writer response==========
I have received your input, and we will contact you later!
Hi! How can I help you? 
I want to code!
==========programmer response==========
print("a = 100")
Hi! How can I help you? 
exit
==========chatbot response==========
GoodBye!


## Markov Text Generator

In [ ]:
import random

class MarkovTextGenerator:
    def __init__(self, text):
        self.model = {}
        words = text.split()
        for i in range(len(words) - 1):
            key = words[i]
            if key not in self.model:
                self.model[key] = []
            self.model[key].append(words[i + 1])

    def generate(self, start_word, length=10):
        result = [start_word]
        for _ in range(length):
            next_word = random.choice(self.model.get(result[-1], ["END"]))
            if next_word == "END":
                break
            result.append(next_word)
        return " ".join(result)

text = "AI is amazing. AI is the future. AI can generate text."
generator = MarkovTextGenerator(text)
print(generator.generate("AI"))

AI is the future. AI can generate text.


## Tool using

In [ ]:
from openai import OpenAI

client = OpenAI()

def get_weather(location):
    weather_data = {
        "Paris, France": {"temperature": "15°C", "condition": "Cloudy"},
        "New York, USA": {"temperature": "22°C", "condition": "Sunny"},
        "Beijing, China": {"temperature": "10°C", "condition": "Smoggy"}
    }
    return weather_data.get(location, {"temperature": "Unknown", "condition": "Unknown"})

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

tool_calls = completion.choices[0].message.tool_calls
if tool_calls:
    for call in tool_calls:
        function_name = call.function.name
        arguments = json.loads(call.function.arguments)

        if function_name == "get_weather":
            location = arguments["location"]
            weather_info = get_weather(location)
            print(weather_info)


{'temperature': '15°C', 'condition': 'Cloudy'}


## example

In [ ]:
class TaskManager:
  def __init__(self, job):
    self.task = {}
    self.team = {}
    self.job = job

  def add_task(self, task_id, task_type):
    self.task[task_id] = task_type

  def process_tasks(self):
    for task_id, task_type in self.task.items():
      self.team[task_type].task[task_id] = task_type
      print(f"Task Manager has already assigned task {task_id} to {task_type} processor.")


class Processor:
  def __init__(self, job):
    self.job = job
    self.task = {}
    self.team = {}

  def process_tasks(self):
    if self.job == 'text':
      for task_id, task_type in self.task.items():
        print(f'Process text task {task_id}')
    else:
      for task_id, task_type in self.task.items():
        print(f'Process image task {task_id}')

class Engine:
  def __init__(self):
    self.team = {}

  def add_agent(self, agent):
    self.team[agent.job] = agent

  def add_friends(self, relationship):
    for me, friend in relationship.items():
      for f in friend:
        self.team[me].team[f] = self.team[f]

  def run(self):
    # while self.team['manager'].task:
    self.team['manager'].process_tasks()
    for job, agent in self.team['manager'].team.items():
      self.team[job].process_tasks()


In [ ]:
relationship = {'manager': ['text', 'image']}
tm = TaskManager('manager')
tx = Processor('text')
ig = Processor('image')

tm.add_task(1, "text")
tm.add_task(2, "image")
tm.add_task(3, "text")

engine = Engine()
engine.add_agent(tm)
engine.add_agent(tx)
engine.add_agent(ig)

engine.add_friends(relationship)
engine.run()

Task Manager has already assigned task 1 to text processor.
Task Manager has already assigned task 2 to image processor.
Task Manager has already assigned task 3 to text processor.
Process text task 1
Process text task 3
Process image task 2
